In [1]:
from compute import Var, eqvar, ureg, get_unit, buildidpvars, coupled_run, Evaluable, args_in_order, fill_args
from inputresolver import reassigneq, eqvars, eqsonly, default_out, mdf_order, default_in, idx_eqlist
from inputresolver import getallvars, invert_edges, resolve, reassign
from representations import bipartite_repr, drawbipartite, digraph_repr, draw
from compute_utils import get_outputs, check_eqs, print_out
from pprint import pprint
import networkx as nx
import sympy as sp
from sympy.utilities.lambdify import implemented_function
import numpy as np
from scipy import interpolate
import openmdao.api as om
from collections import OrderedDict
import matplotlib.pyplot as plt

Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


In [114]:
import scipy.optimize
from functools import reduce

In [10]:
x = Var('x', 2)
a = Var('a')
c = Var('c')
y, eq0 = eqvar('y', x**2)
#y.varval = 10
b, eq1 = eqvar('b', 2*a-x)
eq2 = (c, b**2)
eq3 = (a, c*b)
eq_list = [eq1, eq2, eq3]

In [11]:
scipy.optimize.fsolve(lambda x: np.array([x[1]-(2*x[0]-2), x[2]-x[1]**2, x[0]-x[2]*x[1]]), np.array([1,1,1]))

array([1.58268652, 1.16537304, 1.35809433])

In [12]:
eqs = idx_eqlist(eq_list)
eqv = eqvars(eqs)
dins = default_in(eqs)
dout = default_out(eqs)

In [13]:
prob = om.Problem()
model = prob.model
buildidpvars(dins, model)
counter = coupled_run(eqs, [], (0,1,2), model, model, maxiter=100)
prob.setup();

eq0 b 2*a - x None
eq1 c b**2 None
eq2 a b*c None


In [14]:
prob.run_model()


group1
b [0.]
c [1.]
a [1.]
b [1.2]
c [1.44]
a [1.68]
b [1.16613757]
c [1.35987682]
a [1.58446628]
b [1.16537337]
c [1.35809509]
a [1.58268717]
b [1.16537304]
c [1.35809433]
a [1.58268652]
NL: Newton Converged in 4 iterations


In [15]:
check_eqs(eqs,get_outputs(eqs, model))

{0: (1.1653730430624656, 1.16537304306247),
 1: (1.3580943294965648, 1.35809432949667),
 2: (1.5826865215312327, 1.58268652153129)}

In [17]:
order = ([1,2],3)

In [18]:
order

([1, 2], 3)

In [21]:
r = [left-right for left, right in eqs.values()]

In [23]:
ev = [Evaluable.fromsympy(elt) for elt in r]

In [36]:
ex = [Evaluable.fromsympy(right) for left, right in eqs.values()]

In [58]:
default_in([eqs[elt] for elt in [0,1]], eqdictin=False)

{a, x}

In [61]:
default_in(eqs)

{x}

In [148]:
def resolve_eliminated_implicit(eqs, elim_order, impl_eqs):
    inp = default_in(eqs)
    sols = {elt.name: elt.varval for elt in inp}
    expected_ex = default_in([eqs[elt] for elt in elim_order], eqdictin=False)
    ex_out = {eqs[elt][0] for elt in elim_order}
    expected_im = reduce(lambda a,b: a.union(b), [eqs[elt][1].free_symbols for elt in impl_eqs])
    inp_impl = expected_ex.union(expected_im) -ex_out - inp
    inp_impl_idx = list(enumerate(inp_impl))
    print(inp_impl_idx)
    r = [left-right for left, right in eqs.values()]
    ev = [Evaluable.fromsympy(elt) for elt in r]
    def only_a(a):
        sols.update({elt.name: a[idx] for idx,elt in inp_impl_idx})
        for elt in elim_order:
            sols.update({eqs[elt][0].name : ex[elt].evaldict(sols)})
        return  np.array([ev[evi].evaldict(sols) for evi in impl_eqs])
    return only_a

In [157]:
only_a = resolve_eliminated_implicit(eqs, [0], [1,2])

[(0, c), (1, a)]


In [155]:
only_a([0.1])

array([5.932])

In [160]:
scipy.optimize.fsolve(only_a, np.array([1., 1.]))

array([1.35809433, 1.58268652])

In [48]:
f = lambda inp: np.array([ev[2].evaldict({'x':x.varval, 'a':inp[0], 'b':inp[1], 'c':inp[2]}) for evi in ev])

{'x': 2, 'a': 0.1, 'b': -1.8, 'c': 3.24}

In [34]:
f = lambda inp: np.array([evi.evaldict({'x':x.varval, 'a':inp[0], 'b':inp[1], 'c':inp[2]}) for evi in ev])

In [35]:
f(np.array([0.1, 0.2, 0.3]))

array([2.  , 0.26, 0.04])